In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import logging
import penaltyblog as pb
from types import SimpleNamespace
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
import sys
sys.path.append('../src')
from dixon_coles import create_team_ratings, simulate_league, process_simulation_results
from scraping_utils import initialize_webdriver, scroll_page, get_results, get_fixtures, get_table

# Fetch data

In [ ]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp25/results') # all results
scroll_page(driver)

In [ ]:
df = get_results(driver)
df['season'] = '2025'
df['league_division'] = '7'
df

In [ ]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp24/results') # all results
scroll_page(driver)

In [ ]:
aux = get_results(driver)
aux['season'] = '2024'
aux['league_division'] = '7'
aux

In [ ]:
df = pd.concat([df, aux], ignore_index=True)
df

In [ ]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp23/results') # all results
scroll_page(driver)

In [ ]:
aux = get_results(driver)
aux['season'] = '2023'
aux['league_division'] = '7'
aux

In [ ]:
df = pd.concat([df, aux], ignore_index=True)
df

In [ ]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M6!etejp25/results') # all results
scroll_page(driver)

In [ ]:
aux = get_results(driver)
aux['season'] = '2025'
aux['league_division'] = '6'
aux

In [ ]:
df = pd.concat([df, aux], ignore_index=True)
df

In [ ]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M6!etejp24/results') # all results
scroll_page(driver)

In [ ]:
aux = get_results(driver)
aux['season'] = '2024'
aux['league_division'] = '6'
aux

In [ ]:
df = pd.concat([df, aux], ignore_index=True)
df

In [ ]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M6!etejp23/results') # all results
scroll_page(driver)

In [ ]:
aux = get_results(driver)
aux['season'] = '2023'
aux['league_division'] = '6'
aux

In [ ]:
df = pd.concat([df, aux], ignore_index=True)
df

In [ ]:
df.to_csv('../data/results/finland_6_7_divisions_results_2023_2025.csv', index=False)

# Fetch more data

In [3]:
df = pd.read_csv('../data/results/finland_6_7_divisions_results_2023_2025.csv')
display(df.head())
display(df.tail())
display(df.shape)

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7


,team_home,team_away,goals_home,goals_away,season,league_division
2814,AC StaSi,PK-35/Legendat,3,0,2023,6
2815,ToTe/Keparoi,Ponnistus/Kermaa,3,2,2023,6
2816,Kullervo/Mosaic,HeMan,0,7,2023,6
2817,Ponnistus/Peruskallio,Käpylä Maanantai,0,5,2023,6
2818,HDS/Express,Trikiinit,5,1,2023,6


(2819, 6)

In [4]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp22/results') # all results
scroll_page(driver)

In [5]:
aux = get_results(driver)
aux['season'] = '2022'
aux['league_division'] = '7'
aux

,team_home,team_away,goals_home,goals_away,season,league_division
0,OT-77,HeKuLa,4,0,2022,7
1,FC POHU/Kova Kamppi,PiTa,2,2,2022,7
2,Valtti/Juniors,OT-77,1,1,2022,7
3,AC StaSi/Europort,FC POHU/Kova Kamppi,1,0,2022,7
4,Cosmos,FC Lähiö,1,3,2022,7
...,...,...,...,...,...,...
176,OT-77,AC StaSi/Europort,0,0,2022,7
177,FC Lähiö,Kullervo,1,3,2022,7
178,FC POHU/Swigu,Kilo IF/2,2,2,2022,7
179,Käpylän Sekunda,HeKuLa,0,1,2022,7


In [6]:
df = pd.concat([df, aux], ignore_index=True)
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
2995,OT-77,AC StaSi/Europort,0,0,2022,7
2996,FC Lähiö,Kullervo,1,3,2022,7
2997,FC POHU/Swigu,Kilo IF/2,2,2,2022,7
2998,Käpylän Sekunda,HeKuLa,0,1,2022,7


In [7]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp21/results') # all results
scroll_page(driver)

In [8]:
aux = get_results(driver)
aux['season'] = '2021'
aux['league_division'] = '7'
aux

,team_home,team_away,goals_home,goals_away,season,league_division
0,Roxbury JK Hki,OT-77,0,4,2021,7
1,Roxbury JK Hki,Käpylän Sekunda,2,1,2021,7
2,AC StaSi/Europort,HeKuLa,3,1,2021,7
3,Cosmos,FC Puimur,0,1,2021,7
4,Kullervo,FC POHU/Swigu,1,3,2021,7
...,...,...,...,...,...,...
124,Kullervo,KOPSE/CRPS FC,2,1,2021,7
125,MoPo,Cosmos,2,2,2021,7
126,HeKuLa,Käpylän Sekunda,0,1,2021,7
127,FC Lähiö,FC Pakila,5,3,2021,7


In [9]:
df = pd.concat([df, aux], ignore_index=True)
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
3124,Kullervo,KOPSE/CRPS FC,2,1,2021,7
3125,MoPo,Cosmos,2,2,2021,7
3126,HeKuLa,Käpylän Sekunda,0,1,2021,7
3127,FC Lähiö,FC Pakila,5,3,2021,7


In [10]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp20/results') # all results
scroll_page(driver)

In [11]:
aux = get_results(driver)
aux['season'] = '2020'
aux['league_division'] = '7'
aux

,team_home,team_away,goals_home,goals_away,season,league_division
0,FC POHU/Swigu,Cosmos,1,1,2020,7
1,HJK/Töölön Ihme,FC Kontu/TDJ,0,1,2020,7
2,Ponnistus/TK,HeKuLa,1,1,2020,7
3,HeKuLa,FC POHU/Swigu,0,3,2020,7
4,Roxbury JK Hki,FC POHU/Kova Kamppi,4,0,2020,7
...,...,...,...,...,...,...
129,OT-77,PiTa,1,3,2020,7
130,KOPSE/CRPS FC,FC Puimur,1,4,2020,7
131,Cosmos,KOPSE/CRPS FC,3,1,2020,7
132,AS RaPe,KOPSE/CRPS FC,2,0,2020,7


In [12]:
df = pd.concat([df, aux], ignore_index=True)
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
3258,OT-77,PiTa,1,3,2020,7
3259,KOPSE/CRPS FC,FC Puimur,1,4,2020,7
3260,Cosmos,KOPSE/CRPS FC,3,1,2020,7
3261,AS RaPe,KOPSE/CRPS FC,2,0,2020,7


In [13]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M6!etejp22/results') # all results
scroll_page(driver)

In [14]:
aux = get_results(driver)
aux['season'] = '2022'
aux['league_division'] = '6'
aux

,team_home,team_away,goals_home,goals_away,season,league_division
0,Millbrook FC,FC Pehmeet ja Märät,1,1,2022,6
1,FC Kirkkonummi/4,FC Pehmeet ja Märät,1,3,2022,6
2,ToBK,Millbrook FC,1,3,2022,6
3,ToTe/Tapio,Millbrook FC,4,1,2022,6
4,FC Kirkkonummi/4,FC Dons,0,1,2022,6
...,...,...,...,...,...,...
722,Pelikaani,AC JazzPa,3,1,2022,6
723,Kasiysi/Rovers,FC Kirkkonummi/4,0,4,2022,6
724,KoPa,FC Dons,0,3,2022,6
725,PK-35/Legendat,FC Vantaa,0,2,2022,6


In [15]:
df = pd.concat([df, aux], ignore_index=True)
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
3985,Pelikaani,AC JazzPa,3,1,2022,6
3986,Kasiysi/Rovers,FC Kirkkonummi/4,0,4,2022,6
3987,KoPa,FC Dons,0,3,2022,6
3988,PK-35/Legendat,FC Vantaa,0,2,2022,6


In [16]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M6!etejp21/results') # all results
scroll_page(driver)

In [17]:
aux = get_results(driver)
aux['season'] = '2021'
aux['league_division'] = '6'
aux

,team_home,team_away,goals_home,goals_away,season,league_division
0,MLHF,Purha/2,5,0,2021,6
1,GrIFK/2,KJS,6,0,2021,6
2,FC POHU/KY United,MLHF,3,1,2021,6
3,LePa/2,KJS,5,1,2021,6
4,GrIFK/2,FC Viikingit/B,1,3,2021,6
...,...,...,...,...,...,...
598,FC Kapina/2,KP-75/SaPo,0,10,2021,6
599,HeMan,FC Spital,1,2,2021,6
600,PK-35/Legendat,PETO/2,1,2,2021,6
601,Tavastia/FC Mojo,FC POHU/Shrimps,2,0,2021,6


In [18]:
df = pd.concat([df, aux], ignore_index=True)
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
4588,FC Kapina/2,KP-75/SaPo,0,10,2021,6
4589,HeMan,FC Spital,1,2,2021,6
4590,PK-35/Legendat,PETO/2,1,2,2021,6
4591,Tavastia/FC Mojo,FC POHU/Shrimps,2,0,2021,6


In [19]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M6!etejp20/results') # all results
scroll_page(driver)

In [20]:
aux = get_results(driver)
aux['season'] = '2020'
aux['league_division'] = '6'
aux

,team_home,team_away,goals_home,goals_away,season,league_division
0,FC Villisiat,JaVo,5,3,2020,6
1,LePa/3,Geishan Pallo,0,5,2020,6
2,LJS/Reservi,ViTa,1,2,2020,6
3,Tikka/2,LoPa/2,5,0,2020,6
4,Ruisku/Juniors,KOPSE/NMI,3,1,2020,6
...,...,...,...,...,...,...
541,FC POHU/Atl Koffari,FC Helsinki 2020,4,0,2020,6
542,Kilo IF/2,KeiKa,3,4,2020,6
543,FC Babylon/2,LePa/3,5,1,2020,6
544,HooGee/Hodari,FC Slimmit,1,1,2020,6


In [21]:
df = pd.concat([df, aux], ignore_index=True)
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
5134,FC POHU/Atl Koffari,FC Helsinki 2020,4,0,2020,6
5135,Kilo IF/2,KeiKa,3,4,2020,6
5136,FC Babylon/2,LePa/3,5,1,2020,6
5137,HooGee/Hodari,FC Slimmit,1,1,2020,6


In [22]:
df.to_csv('../data/results/finland_6_7_divisions.csv', index=False)

# Data processing

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5139 entries, 0 to 5138
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   team_home        5139 non-null   object
 1   team_away        5139 non-null   object
 2   goals_home       5139 non-null   object
 3   goals_away       5139 non-null   object
 4   season           5139 non-null   object
 5   league_division  5139 non-null   object
dtypes: object(6)
memory usage: 241.0+ KB


In [ ]:
# fix dtypes
df[['goals_home', 'goals_away', 'season', 'league_division']] = df[['goals_home', 'goals_away', 'season', 'league_division']].astype(int)

In [38]:
team_name = 'Trikiinit'
df[(df['team_home']==team_name) | (df['team_away']==team_name)]

,team_home,team_away,goals_home,goals_away,season,league_division
10,Trikiinit,PK-35/Äijät,6,0,2025,7
12,Valtti/4,Trikiinit,2,5,2025,7
29,MLHF/Reserves,Trikiinit,1,5,2025,7
37,Trikiinit,Kullervo,4,1,2025,7
58,Trikiinit,Cosmos,2,1,2025,7
...,...,...,...,...,...,...
4877,Trikiinit,Ponnistus/Peruskallio,2,0,2020,6
4912,LPS/2,Trikiinit,5,0,2020,6
5031,HePuLi,Trikiinit,4,0,2020,6
5090,Trikiinit,Ruila,3,2,2020,6


In [39]:
# Group by "team_home" and calculate the set of league_division values
team_divisions = df.groupby("team_home")["league_division"].apply(set)

# Filter teams where the number of unique league_division values is greater than 1
teams_with_multiple_divisions = team_divisions[team_divisions.apply(len) > 1]

# Get the team names
teams = teams_with_multiple_divisions.index.tolist()

print("Teams that have played in more than one division:")
print(teams)

Teams that have played in more than one division:
['AFC EMU', 'FC Kontu/TDJ', 'FC Puimur', 'HJK/Töölön Ihme', 'Kilo IF/2', 'Kullervo/Überkleber', 'PiTa', 'PiTa/II', 'Tavastia/FC Mojo', 'Trikiinit', 'Valtti/3']


In [40]:
print('Number of teams in total:')
print(df['team_home'].nunique())

Number of teams in total:
281


In [57]:
team_name = 'Trikiinit'
df[(df['team_home']==team_name) | (df['team_away']==team_name)].groupby(['season'])['league_division'].unique().reset_index().sort_values(by='season')


,season,league_division
0,2020,[6]
1,2021,[7]
2,2022,[6]
3,2023,[6]
4,2024,[7]
5,2025,[7]
